# <div align="center"> Battle of the Neighborhoods - Toronto<div align="center">

By Irina Safronova

Questions 1, 2 and 3 of the assignment are segregated by the headers. 
You can click on the links in the Table of Contents section.

## Table of Contents
1. [Question 1](#Question1)
2. [Question 2](#Question2)
3. [Question 3](#Question3)

## Question 1 <a id="Question1"></a>

Before working with the data, let's import the dependencies we need. 

In [1]:
import pandas as pd #library for data analysis

In [2]:
!pip install bs4 
from bs4 import BeautifulSoup #library for parsing HTML documents
import requests #library for making HTTP requests

First, we download the contents of the webpage in text format and store it in a variable.

In [3]:
wiki_url = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text

Then, we create a BeautifulSoup object and find the table in the webpage. We'll create a list to store the table information in.

In [4]:
soup = BeautifulSoup(wiki_url, 'html.parser')

In [5]:
table_contents = []
table = soup.find('table')

Let's create a dictionary cell to store the information. The dictionary will have 3 keys: Postal Code, Borough and Neighborhood. We'll drop the cells which do not contain the information.

In [6]:
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['Postal Code'] = row.p.text[:3] #extract 3 characters of the postal code
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

In [7]:
table_contents

[{'Postal Code': 'M3A', 'Borough': 'North York', 'Neighborhood': 'Parkwoods'},
 {'Postal Code': 'M4A',
  'Borough': 'North York',
  'Neighborhood': 'Victoria Village'},
 {'Postal Code': 'M5A',
  'Borough': 'Downtown Toronto',
  'Neighborhood': 'Regent Park, Harbourfront'},
 {'Postal Code': 'M6A',
  'Borough': 'North York',
  'Neighborhood': 'Lawrence Manor, Lawrence Heights'},
 {'Postal Code': 'M7A',
  'Borough': "Queen's Park",
  'Neighborhood': 'Ontario Provincial Government'},
 {'Postal Code': 'M9A',
  'Borough': 'Etobicoke',
  'Neighborhood': 'Islington Avenue'},
 {'Postal Code': 'M1B',
  'Borough': 'Scarborough',
  'Neighborhood': 'Malvern, Rouge'},
 {'Postal Code': 'M3B',
  'Borough': 'North York',
  'Neighborhood': 'Don Mills North'},
 {'Postal Code': 'M4B',
  'Borough': 'East York',
  'Neighborhood': 'Parkview Hill, Woodbine Gardens'},
 {'Postal Code': 'M5B',
  'Borough': 'Downtown Toronto',
  'Neighborhood': 'Garden District, Ryerson'},
 {'Postal Code': 'M6B', 'Borough': 'Nort

Now we'll create a dataframe. We also need to replace some of the boroughs' names.

In [8]:
csv_data = pd.DataFrame(table_contents)
csv_data['Borough'] = csv_data['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})

In [9]:
csv_data.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


Let's have a look at the dataframe shape:

In [10]:
print ('Answer to Question 1: the dataframe has {} rows and {} columns.'.format(csv_data.shape[0], csv_data.shape[1]))

Answer to Question 1: the dataframe has 103 rows and 3 columns.


## Question 2 <a id="Question2"></a>

Let's install the geocoder library.

The approach suggested in the guidelines didn't work, so we are going to import the data from a .csv file.

#initialize your variable to None
lat_lng_coords = None

#loop until you get the coordinates
while(lat_lng_coords is None):
  g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
  lat_lng_coords = g.latlng

latitude = lat_lng_coords[0]
longitude = lat_lng_coords[1]

In [11]:
wiki_data = pd.read_csv("https://cocl.us/Geospatial_data")
wiki_data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


We need to look at the dataframes' shape to determine if and how we can join them.

In [12]:
print("The shape of the wiki data is:", csv_data.shape)
print("The shape of the csv data is:", wiki_data.shape)

The shape of the wiki data is: (103, 3)
The shape of the csv data is: (103, 3)


Since the dimensions are the same, we are going to perform an inner join on Postal Code.

Let's check the column types of both dataframes to check if the Postal Code columns are of the same type.

In [13]:
csv_data.dtypes

Postal Code     object
Borough         object
Neighborhood    object
dtype: object

In [14]:
wiki_data.dtypes

Postal Code     object
Latitude       float64
Longitude      float64
dtype: object

In [15]:
combined_data = csv_data.join(wiki_data.set_index('Postal Code'), on='Postal Code', how='inner')

In [16]:
combined_data.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


In [17]:
print ('Answer to Question 2: the dataframe has {} rows and {} columns.'.format(combined_data.shape[0], combined_data.shape[1]))

Answer to Question 2: the dataframe has 103 rows and 5 columns.


As the result of the inner join, we have got 103 rows, which means it was succesful.

## Question 3 <a id="Question3"></a>

We are going to cluster Toronto based on similarities between the neighborhoods using k-means clustering and Forsquare API.

First, let's install and import the dependencies we need.

In [18]:
!pip install geopy
from geopy.geocoders import Nominatim

!pip install folium
import folium

import numpy as np

from sklearn.cluster import KMeans

import matplotlib.cm as cm
import matplotlib.colors as colors

In [19]:
address = "Toronto, Ontario"

geolocator = Nominatim(user_agent="toronto_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

43.6534817 -79.3839347


Let's visualize the map of Toronto.

In [20]:
toronto_map = folium.Map(location=[latitude, longitude], zoom_start=13) 

for latitude, longitude, borough, neighborhood in zip(combined_data['Latitude'], combined_data['Longitude'], 
                           combined_data['Borough'], combined_data['Neighborhood']):
    label = '{}.{}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html="True")
    folium.CircleMarker(
        [latitude, longitude],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(toronto_map)

# display map
toronto_map

We initialize Foursquare API credentials. 

In [21]:
CLIENT_ID = 'JIUGE3RHXKOY2AY1LM5CKGKNE2OSFIPWX0NAI2TKQZTYQJH2' 
CLIENT_SECRET = 'OCKZXOPOYCN2Y40ESWYY3C1CWQQ5AMQSMSYOIT1J00AU1PXM' 
ACCESS_TOKEN = 'DHVLK51X4HNRQ3EJLTXOSJ2NVDB3JA3TTSXCFN1ZY4P3FQY2'
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: JIUGE3RHXKOY2AY1LM5CKGKNE2OSFIPWX0NAI2TKQZTYQJH2
CLIENT_SECRET:OCKZXOPOYCN2Y40ESWYY3C1CWQQ5AMQSMSYOIT1J00AU1PXM


We create a function to get the venue categories in Toronto.

In [22]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Now we will run the function on each neighborhood and store the information in a new dataframe called toronto_venues.

In [23]:
toronto_venues = getNearbyVenues(names=combined_data['Neighborhood'],
                                   latitudes=combined_data['Latitude'],
                                   longitudes=combined_data['Longitude']
                                  )

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Ontario Provincial Government
Islington Avenue
Malvern, Rouge
Don Mills North
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills South
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
The Danforth  East
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmount Park
Bayview Village
Downsview East
The Danforth

Checking the size of the dataframe:

In [24]:
print(toronto_venues.shape)
toronto_venues.head()

(1337, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,KFC,43.754387,-79.333021,Fast Food Restaurant
2,Parkwoods,43.753259,-79.329656,TTC stop #8380,43.752672,-79.326351,Bus Stop
3,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
4,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena


Let's check how many venues were returned for each neighborhood.

In [25]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,5,5,5,5,5,5
"Alderwood, Long Branch",7,7,7,7,7,7
"Bathurst Manor, Wilson Heights, Downsview North",22,22,22,22,22,22
Bayview Village,4,4,4,4,4,4
"Bedford Park, Lawrence Manor East",25,25,25,25,25,25
...,...,...,...,...,...,...
"Willowdale, Newtonbrook",2,2,2,2,2,2
Woburn,4,4,4,4,4,4
Woodbine Heights,8,8,8,8,8,8


How many unique venue categories are there?

In [26]:
print('There are {} unique categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 237 unique categories.


We perform one hot encoding on the venue categories.

In [27]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot

,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1332,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1333,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1334,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1335,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [28]:
# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

,Yoga Studio,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Next, we check the dataframe shape.

In [29]:
toronto_onehot.shape

(1337, 237)

Then, we group rows by neighborhood and by the mean of the frequency of occurence of each category.

In [30]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0
1,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0
3,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0
4,"Bedford Park, Lawrence Manor East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,"Willowdale, Newtonbrook",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0
97,Woburn,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0
98,Woodbine Heights,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.125,0.0,0.0,0.0,0.0,0.0,0.0
99,York Mills West,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0


What is the new dataframe shape?

In [31]:
toronto_grouped.shape

(101, 237)

Let's define a function to sort the venues in descending order to determine which are the most common ones.

In [32]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

As there are too many venue categories, we will take the top 10 for each neighborhood and define a new dataframe to store the information in.

In [33]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Breakfast Spot,Lounge,Clothing Store,Latin American Restaurant,Skating Rink,Yoga Studio,Movie Theater,Men's Store,Metro Station,Mexican Restaurant
1,"Alderwood, Long Branch",Pizza Place,Sandwich Place,Gym,Pub,Coffee Shop,Skating Rink,Middle Eastern Restaurant,Modern European Restaurant,Mobile Phone Shop,Miscellaneous Shop
2,"Bathurst Manor, Wilson Heights, Downsview North",Bank,Coffee Shop,Pizza Place,Diner,Middle Eastern Restaurant,Mobile Phone Shop,Supermarket,Shopping Mall,Sushi Restaurant,Fried Chicken Joint
3,Bayview Village,Bank,Chinese Restaurant,Japanese Restaurant,Café,Yoga Studio,Museum,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant
4,"Bedford Park, Lawrence Manor East",Italian Restaurant,Sandwich Place,Coffee Shop,Pub,Fast Food Restaurant,Butcher,Café,Locksmith,Liquor Store,Sushi Restaurant


Now we are going to create a model to cluster the neighborhoods. We will run k-means and cluster the neighborhoods into 5 clusters.

In [34]:
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

kmeans.labels_ 

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 2, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1,
       2, 4, 1, 1, 1, 4, 1, 4, 1, 1, 1, 4, 1, 4, 1, 4, 1, 1, 2, 1, 1, 1,
       4, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1,
       1, 1, 1, 1, 3, 1, 1, 1, 0, 1, 1, 4, 0])

We create a new dataframe which includes the clusters and top 10 venues for each neighborhood.

In [35]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [36]:
toronto_merged = combined_data

toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_merged.head()
print(toronto_merged.shape)

(103, 16)


To make sure there are no undefined values in the new dataframe, let's drop them and check the shape of the dataframe.

In [37]:
toronto_merged = toronto_merged.dropna(subset=['Cluster Labels'])
toronto_merged.shape

(101, 16)

Finally, let's visualize the clusters on the map. 

In [38]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Let's verify the clusters.

Cluster 1:

In [39]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
45,North York,0.0,Park,Yoga Studio,Movie Theater,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant
52,North York,0.0,Park,Yoga Studio,Movie Theater,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant


Cluster 2:

In [40]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North York,1.0,Portuguese Restaurant,French Restaurant,Intersection,Coffee Shop,Hockey Arena,Mobile Phone Shop,Motel,Monument / Landmark,Modern European Restaurant,Middle Eastern Restaurant
2,Downtown Toronto,1.0,Coffee Shop,Park,Bakery,Breakfast Spot,Pub,Spa,Café,Shoe Store,Chocolate Shop,Restaurant
3,North York,1.0,Clothing Store,Coffee Shop,Event Space,Boutique,Vietnamese Restaurant,Furniture / Home Store,Accessories Store,Men's Store,Metro Station,Mexican Restaurant
4,Queen's Park,1.0,Coffee Shop,Sushi Restaurant,Yoga Studio,College Cafeteria,Spa,Burrito Place,Smoothie Shop,Café,Sandwich Place,College Auditorium
6,Scarborough,1.0,Fast Food Restaurant,Movie Theater,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop
...,...,...,...,...,...,...,...,...,...,...,...,...
97,Downtown Toronto,1.0,Café,Coffee Shop,Restaurant,Hotel,Seafood Restaurant,Pizza Place,Gluten-free Restaurant,Speakeasy,Steakhouse,Tea Room
98,Etobicoke,1.0,Pool,River,Movie Theater,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant
99,Downtown Toronto,1.0,Creperie,Breakfast Spot,Martial Arts School,Escape Room,Bubble Tea Shop,Sushi Restaurant,Beer Bar,Salon / Barbershop,Coffee Shop,Burger Joint
100,East Toronto Business,1.0,Yoga Studio,Restaurant,Fast Food Restaurant,Farmers Market,Burrito Place,Skate Park,Light Rail Station,Garden,Spa,Auto Workshop


Cluster 3:

In [41]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
53,North York,2.0,Baseball Field,Food Truck,Yoga Studio,Motel,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant
57,North York,2.0,Baseball Field,Yoga Studio,Market,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant
101,Etobicoke,2.0,Baseball Field,Yoga Studio,Market,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant


Cluster 4:

In [42]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
64,York,3.0,Convenience Store,Yoga Studio,Museum,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop


Cluster 5:

In [43]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,4.0,Fast Food Restaurant,Food & Drink Shop,Park,Bus Stop,Motel,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant
10,North York,4.0,Japanese Restaurant,Playground,Park,Bakery,Movie Theater,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant
16,York,4.0,Field,Hockey Arena,Park,Trail,Motel,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station
21,York,4.0,Park,Women's Store,Pool,Wine Shop,Malay Restaurant,Martial Arts School,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store
35,East York/East Toronto,4.0,Coffee Shop,Convenience Store,Park,Yoga Studio,Movie Theater,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant
40,North York,4.0,Airport,Park,Yoga Studio,Movie Theater,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant
49,North York,4.0,Park,Bakery,Construction & Landscaping,Trail,Basketball Court,Yoga Studio,Modern European Restaurant,Mobile Phone Shop,Miscellaneous Shop,Middle Eastern Restaurant
61,Central Toronto,4.0,Dim Sum Restaurant,Swim School,Park,Bus Line,Motel,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant
66,North York,4.0,Convenience Store,Park,Yoga Studio,Movie Theater,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant
77,Etobicoke,4.0,Park,Mobile Phone Shop,Sandwich Place,Yoga Studio,Motel,Massage Studio,Medical Center,Mediterranean Restaurant,Men's Store,Metro Station


We have clustered Toronto neighborhoods based on venue categories.